The mnist digit recognition is the very first algorithm to write for machine learning

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
%matplotlib inline

In [ ]:
mnist = input_data.read_data_sets('data/mnist_data/',one_hot=True)

See what was downloaded

In [ ]:
%ls data/mnist_data

Access the data and see what's in it

In [ ]:
training_digits,training_labels = mnist.train.next_batch(5000)
test_digits, test_labels = mnist.test.next_batch(200)

In [ ]:
print(training_digits.shape)
print(training_labels.shape)
print(training_digits,training_labels)


These data will be used as inputs to the model , so let's create placeholders for them, as well as :
1. Define the cost
2. Create optimizer, that  minimizes this cost
3. Batch up the training set and looping through it, while fitting the parameters W and b by running the optimizer

In [ ]:

learning_rate=0.01
batch_size=100
display_step = 1
mnist = input_data.read_data_sets('data/mnist_data/',one_hot=True)

X=tf.placeholder(dtype=tf.float32,shape=[None,784], name='X')
Y=tf.placeholder(dtype=tf.float32,shape=[None,10], name='Y')

W = tf.Variable(tf.zeros([784,10]),name='W')
b= tf.Variable(tf.zeros([10]),name='b')


Z=tf.matmul(X,W) +b
A=tf.nn.softmax(Z)

cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(A)))
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    batch_index=0
    total_batch = int(mnist.train.num_examples/batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        _, batch_cost = sess.run([train_step, cost], feed_dict={X:batch_xs,Y:batch_ys}) 
        batch_index +=1
        #print("Batch:", '%04d' % (batch_index), "cost=", "{:.9f}".format(batch_cost))                        
    print("Optimization Finished!")     
    
    # Test model
    correct_prediction = tf.equal(tf.argmax(A, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    print("Train Accuracy:", accuracy.eval({X: mnist.train.images, Y: mnist.train.labels}))
    print("Test Accuracy:", accuracy.eval({X: mnist.test.images, Y: mnist.test.labels}))
    
    image=mnist.train.images[600]
    result=sess.run(tf.argmax(A,axis=1),feed_dict={X:image.reshape(1,784)})
    print("The recognized digit is {}".format(result))
    plt.imshow(image.reshape(28,28))
    
writer = tf.summary.FileWriter('./tbout/mnist1',sess.graph)
writer.close()                

# Cross Entropy Cost Function

We've got to abandon the Mean Squared cost function because it slows down learning. We want a cost function that that learns quickly when the model is a lot wrong and slows down when the model is not that wrong

Here's the best explaination for this - 

http://neuralnetworksanddeeplearning.com/chap3.html#the_cross-entropy_cost_function

Entropy - disorder, uncertainty. If there is no uncertainty there is no information, e.g. if it always is sunny, if the weather forecast says it's sunny - there is not much information gained from it. 
    
    `Entropy = amount of information.`  
 


E.g. 

Assuming a known distribution of AB - 100% A and 0% B, how much information do you get when I tell you it's A?
    
    You've always know it'll be A because the distribution is 100% A, so it's not a surprise it's A, therefore you get 0 bit of information
    
    
Assuming a known distribution of AB - 50% A and 50% B, how much information do you get when I tell you it's A?
    
    It could have be A or B - if I predicted it I could only gotten it right half the time - I've got to ask one question before I am certain ( is it A or B ). So I've got 1 bit of information 
        
    
Assuming a known distribution of ABC&D - 25% of A, 25% of B, 25% of C and 25% D, 
    
    I could only have get a right outcome for sure after 2 questions, e.g. 
    
    1-Is it A or B? 
        Yes - is it A? 
              Yes - it's A 
              No - it's B
        No - is it C?
             Yes - it's C
             No - it's D
             
    I get 2 bit of information.
    
Assuming a know weather distribution of 75% sunny and 25% rainy,

    When it's forecasted that it's going to rain tomorrow, my uncertainty has been reduced by a factor of 4(1/0.25) or I get 2 bit of information also give by log2 4 or -log2 p
  
What is the amount information you'll get from the weather forecast on average ?

    -0.25*log2(0.25)-0.75log2(0.75)=0.81

This is entropy, in general the entropy is, 
    
    `H(p)=-np.sum(p*np.log2(p))`

It's a measure of average information received or the unpredictability of the probablity distribution

Cross entropy refers to the entropy calculated using the true distribution and the predicted distribution :

    `H(p,q)=-np.sum(p*np.log2(q))` - p - true distribution and q - predicted distribution
    
Why do we use it as a cost function? It's a guess, but H(p) = H(p,q) if p=q, when it's not H(p,q) > H(p), and the differenceis  called the kullback leibler divergence(KLD), and I read somewhere that KLD provides some sort of distance measure between the true and the predicted distribution.
    
https://www.youtube.com/watch?v=ErfnhcEV1O8

# Saving the parameters ( variables in tensorflow )

We'd want to be able to save the parameters and then use them for predictions, so let's train the model and save it here


In [ ]:
tf.reset_default_graph()
learning_rate=0.01
batch_size=100
display_step = 1
X=tf.placeholder(dtype=tf.float32,shape=[None,784], name='X')
Y=tf.placeholder(dtype=tf.float32,shape=[None,10], name='Y')

W = tf.Variable(tf.zeros([784,10]),name='W')
b= tf.Variable(tf.zeros([10]),name='b')

Z=tf.matmul(X,W) +b
A=tf.nn.softmax(Z)
mnist = input_data.read_data_sets('data/mnist_data/',one_hot=True)

cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(A), axis=-1))

train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    batch_index=0
    total_batch = int(mnist.train.num_examples/batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        _, batch_cost = sess.run([train_step, cost], feed_dict={X:batch_xs,Y:batch_ys}) 
        batch_index +=1
        #print("Batch:", '%04d' % (batch_index), "cost=", "{:.9f}".format(batch_cost))                        
            
    print("Optimization Finished!")     
   
    # Test model
    correct_prediction = tf.equal(tf.argmax(A, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    print("Train Accuracy:", accuracy.eval({X: mnist.train.images, Y: mnist.train.labels}))
    print("Test Accuracy:", accuracy.eval({X: mnist.test.images, Y: mnist.test.labels}))
    saver = tf.train.Saver() 
    model_path=saver.save(sess, "./tbout/models/model.ckpt")
    print ( 'Saving model to {}'.format(model_path) )
    
writer = tf.summary.FileWriter('./tbout/mnist2',sess.graph)
writer.close()    

Now let's retrieve it and use it for prediction

In [ ]:
tf.reset_default_graph()
W = tf.get_variable("W", shape=[784,10])
b = tf.get_variable("b", shape=[10])
X=tf.placeholder(dtype=tf.float32,shape=[None,784], name='X')
Y=tf.placeholder(dtype=tf.float32,shape=[None,10], name='Y')
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    saver = tf.train.Saver() 
    saver.restore(sess, "./tbout/models/model.ckpt")

    print(b.eval())

    Z=tf.matmul(X,W) +b
    A=tf.nn.softmax(Z)

    image=mnist.train.images[600]
    result=sess.run(tf.argmax(A,axis=1),feed_dict={X:image.reshape(1,784)})
    print("The recognized digit is {}".format(result))
    plt.imshow(image.reshape(28,28))
